In [85]:
# compress analysis
import glob, json, os, re, csv

# results dir
dir = "results/compress"

# devices
devices = ["bf2"]

# Dictionary to store results indexed by (i, j)
results = {device: {} for device in devices}
reconfiguration_results = {device: {} for device in devices}

# averages for plotting
averaged_results = {device: {} for device in devices}

# Pattern to match results
cpu_pattern = re.compile(r"results-(\d+)-(\d+)-(.+)-cpu-compress\.json")
doca_pattern = re.compile(r"results-(\d+)-(\d+)-(.+)-doca-compress\.json")

# Load all matching JSON files
for device in devices:
    for file in glob.glob(f"{dir}/{device}/results-*-*-*-cpu-compress.json"):
        match = cpu_pattern.match(os.path.basename(file))
        if match:
            i, j, filename = match.groups()
            i, j = int(i), int(j)  # Convert i, j to integers

            # Read JSON content
            with open(file, "r") as f:
                data = json.load(f)

            with open(f"{dir}/{device}/results-{i}-{j}-{filename}.size", 'r') as ssize:
                full_file_size = int(ssize.readline().strip().split()[-1])

            # Store in results dictionary, use doca as "sharing percentage"
            key = j
            if key not in results[device]:
                results[device][key] = []

            if key not in reconfiguration_results[device]:
                    reconfiguration_results[device][key] = []

            joined_runtime_seconds = float(data["joined_submission_elapsed"])

            if 0 < i < 100:
                with open(f"{dir}/{device}/results-{i}-{j}-{filename}-doca-compress.json", "r") as f:
                    data = json.load(f)
                
                reconfiguration_runtime_seconds = float(data["overall_submission_elapsed"]) + float(data["ctx_stop_elapsed"])
                reconfiguration_runtime_seconds = max(joined_runtime_seconds, reconfiguration_runtime_seconds)
                reconfiguration_results[device][key].append((full_file_size / 1_048_576) / reconfiguration_runtime_seconds)
            elif i == 100:
                reconfiguration_results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)

            results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)
    
    # no cpu file, only dpu
    for file in glob.glob(f"{dir}/{device}/results-0-100-*-doca-compress.json"):
        match = doca_pattern.match(os.path.basename(file))
        if match:
            i, j, filename = match.groups()
            i, j = int(i), int(j)  # Convert i, j to integers
 
            # Read JSON content
            with open(file, "r") as f:
                data = json.load(f)

            with open(f"{dir}/{device}/results-{i}-{j}-{filename}.size", 'r') as ssize:
                full_file_size = int(ssize.readline().strip().split()[-1])

            # Store in results dictionary, use doca as "sharing percentage"
            key = j
            if key not in results[device]:
                results[device][key] = []

            joined_runtime_seconds = float(data["joined_submission_elapsed"])
            results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)

            if key not in reconfiguration_results[device]:
                reconfiguration_results[device][key] = []

            reconfiguration_runtime_seconds = float(data["overall_submission_elapsed"]) + float(data["ctx_stop_elapsed"])
            reconfiguration_runtime_seconds = max(joined_runtime_seconds, reconfiguration_runtime_seconds)
            reconfiguration_results[device][key].append((full_file_size / 1_048_576) / reconfiguration_runtime_seconds)

    averaged_results[device] = {key: {"static": 0, "reconfiguration": 0} for key in results[device].keys()}
    for key in results[device].keys():
        key_avg = sum(results[device][key]) / len(results[device][key])
        averaged_results[device][key]["static"] = key_avg
        reconfiguration_key_avg = sum(reconfiguration_results[device][key]) / len(reconfiguration_results[device][key])
        averaged_results[device][key]["reconfiguration"] = reconfiguration_key_avg

display(averaged_results["bf2"])

# Write to CSV
with open(f"../../tex/figures/za-evaluation/coprocessing-compress-deflate.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    writer.writerow(["sharing_percentage", "static", "reconfiguration"])

    # Write rows
    sorted_keys = sorted(averaged_results["bf2"].keys())
    for key in sorted_keys: # only bf2 has this item
        writer.writerow([key, averaged_results["bf2"][key]["static"], averaged_results["bf2"][key]["reconfiguration"]])


{20: {'static': 31.97834088959665, 'reconfiguration': 30.438697857590885},
 70: {'static': 77.4070359864457, 'reconfiguration': 71.70595817260433},
 90: {'static': 233.43514889216965, 'reconfiguration': 123.48639410822146},
 80: {'static': 114.71725544867532, 'reconfiguration': 95.15381134344318},
 40: {'static': 41.06306195985683, 'reconfiguration': 38.955311952611844},
 60: {'static': 59.25174485264477, 'reconfiguration': 55.251000187669625},
 50: {'static': 48.84624431910509, 'reconfiguration': 46.067005010893524},
 0: {'static': 25.51804771002575, 'reconfiguration': 25.51804771002575},
 10: {'static': 28.31210745602425, 'reconfiguration': 27.778281838499254},
 30: {'static': 36.229392386871496, 'reconfiguration': 34.34155118948264},
 100: {'static': 2754.2790873734593, 'reconfiguration': 141.37024998243638}}

In [86]:
# decompress deflate analysis
import glob, json, os, re, csv

# results dir
dir = "results/decompress-deflate"

# devices
devices = ["bf2"]

# Dictionary to store results indexed by (i, j)
results = {device: {} for device in devices}
reconfiguration_results = {device: {} for device in devices}

# averages for plotting
averaged_results = {device: {} for device in devices}

# Pattern to match results
cpu_pattern = re.compile(r"results-(\d+)-(\d+)-(.+)-cpu-decompress-deflate\.json")
doca_pattern = re.compile(r"results-(\d+)-(\d+)-(.+)-doca-decompress-deflate\.json")

# Load all matching JSON files
for device in devices:
    for file in glob.glob(f"{dir}/{device}/results-*-*-*-cpu-decompress-deflate.json"):
        match = cpu_pattern.match(os.path.basename(file))
        if match:
            i, j, filename = match.groups()
            i, j = int(i), int(j)  # Convert i, j to integers

            # Read JSON content
            with open(file, "r") as f:
                data = json.load(f)

            with open(f"{dir}/{device}/results-{i}-{j}-{filename}.size", 'r') as ssize:
                full_file_size = int(ssize.readline().strip().split()[-1])

            # Store in results dictionary, use doca as "sharing percentage"
            key = j
            if key not in results[device]:
                results[device][key] = []

            if key not in reconfiguration_results[device]:
                    reconfiguration_results[device][key] = []

            joined_runtime_seconds = float(data["joined_submission_elapsed"])

            if 0 < i < 100:
                with open(f"{dir}/{device}/results-{i}-{j}-{filename}-doca-decompress-deflate.json", "r") as f:
                    data = json.load(f)
                
                reconfiguration_runtime_seconds = float(data["overall_submission_elapsed"]) + float(data["ctx_stop_elapsed"])
                reconfiguration_runtime_seconds = max(joined_runtime_seconds, reconfiguration_runtime_seconds)
                reconfiguration_results[device][key].append((full_file_size / 1_048_576) / reconfiguration_runtime_seconds)
            elif i == 100:
                reconfiguration_results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)

            results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)
    
    # no cpu file, only dpu
    for file in glob.glob(f"{dir}/{device}/results-0-100-*-doca-decompress-deflate.json"):
        match = doca_pattern.match(os.path.basename(file))
        if match:
            i, j, filename = match.groups()
            i, j = int(i), int(j)  # Convert i, j to integers
 
            # Read JSON content
            with open(file, "r") as f:
                data = json.load(f)

            with open(f"{dir}/{device}/results-{i}-{j}-{filename}.size", 'r') as ssize:
                full_file_size = int(ssize.readline().strip().split()[-1])

            # Store in results dictionary, use doca as "sharing percentage"
            key = j
            if key not in results[device]:
                results[device][key] = []

            joined_runtime_seconds = float(data["joined_submission_elapsed"])
            results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)

            if key not in reconfiguration_results[device]:
                reconfiguration_results[device][key] = []

            reconfiguration_runtime_seconds = float(data["overall_submission_elapsed"]) + float(data["ctx_stop_elapsed"])
            reconfiguration_runtime_seconds = max(joined_runtime_seconds, reconfiguration_runtime_seconds)
            reconfiguration_results[device][key].append((full_file_size / 1_048_576) / reconfiguration_runtime_seconds)

    averaged_results[device] = {key: {"static": 0, "reconfiguration": 0} for key in results[device].keys()}
    for key in results[device].keys():
        key_avg = sum(results[device][key]) / len(results[device][key])
        averaged_results[device][key]["static"] = key_avg
        reconfiguration_key_avg = sum(reconfiguration_results[device][key]) / len(reconfiguration_results[device][key])
        averaged_results[device][key]["reconfiguration"] = reconfiguration_key_avg

display(averaged_results["bf2"])

# Write to CSV
with open(f"../../tex/figures/za-evaluation/coprocessing-decompress-deflate.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    writer.writerow(["sharing_percentage", "static", "reconfiguration"])

    # Write rows
    sorted_keys = sorted(averaged_results["bf2"].keys())
    for key in sorted_keys: # only bf2 has this item
        writer.writerow([key, averaged_results["bf2"][key]["static"], averaged_results["bf2"][key]["reconfiguration"]])


{40: {'static': 323.40756766274814, 'reconfiguration': 142.8504715684955},
 20: {'static': 232.62281548665692, 'reconfiguration': 128.25695895374042},
 70: {'static': 444.6406759400579, 'reconfiguration': 146.5376473025934},
 90: {'static': 1046.2958408293343, 'reconfiguration': 145.94624957883494},
 60: {'static': 423.46977850475787, 'reconfiguration': 145.5138356034847},
 50: {'static': 353.5021788617539, 'reconfiguration': 148.0697296330436},
 10: {'static': 214.32047108176798, 'reconfiguration': 123.31823058428309},
 80: {'static': 610.4107844513184, 'reconfiguration': 144.88852856666875},
 0: {'static': 219.52897002752616, 'reconfiguration': 219.52897002752616},
 30: {'static': 266.62112481664013, 'reconfiguration': 134.72678541970194},
 100: {'static': 3201.554378106742, 'reconfiguration': 145.21167536260648}}

In [4]:
# decompress LZ4 analysis
import glob, json, os, re, csv

# results dir
dir = "results/decompress-lz4"

# devices
devices = ["bf3"]

# Dictionary to store results indexed by (i, j)
results = {device: {} for device in devices}
reconfiguration_results = {device: {} for device in devices}

# averages for plotting
averaged_results = {device: {} for device in devices}

# Pattern to match results
cpu_pattern = re.compile(r"results-(\d+)-(\d+)-(.+)-cpu-decompress-lz4\.json")
doca_pattern = re.compile(r"results-(\d+)-(\d+)-(.+)-doca-decompress-lz4\.json")

# Load all matching JSON files
for device in devices:
    for file in glob.glob(f"{dir}/{device}/results-*-*-*-cpu-decompress-lz4.json"):
        match = cpu_pattern.match(os.path.basename(file))
        if match:
            i, j, filename = match.groups()
            i, j = int(i), int(j)  # Convert i, j to integers

            # Read JSON content
            with open(file, "r") as f:
                data = json.load(f)

            with open(f"{dir}/{device}/results-{i}-{j}-{filename}.size", 'r') as ssize:
                full_file_size = int(ssize.readline().strip().split()[-1])

            # Store in results dictionary, use doca as "sharing percentage"
            key = j
            if key not in results[device]:
                results[device][key] = []

            if key not in reconfiguration_results[device]:
                    reconfiguration_results[device][key] = []

            joined_runtime_seconds = float(data["joined_submission_elapsed"])

            if 0 < i < 100:
                with open(f"{dir}/{device}/results-{i}-{j}-{filename}-doca-decompress-lz4.json", "r") as f:
                    data = json.load(f)
                
                reconfiguration_runtime_seconds = float(data["overall_submission_elapsed"]) + float(data["ctx_stop_elapsed"])
                reconfiguration_runtime_seconds = max(joined_runtime_seconds, reconfiguration_runtime_seconds)
                reconfiguration_results[device][key].append((full_file_size / 1_048_576) / reconfiguration_runtime_seconds)
            elif i == 100:
                reconfiguration_results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)

            results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)
    
    # no cpu file, only dpu
    for file in glob.glob(f"{dir}/{device}/results-0-100-*-doca-decompress-lz4.json"):
        match = doca_pattern.match(os.path.basename(file))
        if match:
            i, j, filename = match.groups()
            i, j = int(i), int(j)  # Convert i, j to integers
 
            # Read JSON content
            with open(file, "r") as f:
                data = json.load(f)

            with open(f"{dir}/{device}/results-{i}-{j}-{filename}.size", 'r') as ssize:
                full_file_size = int(ssize.readline().strip().split()[-1])

            # Store in results dictionary, use doca as "sharing percentage"
            key = j
            if key not in results[device]:
                results[device][key] = []

            joined_runtime_seconds = float(data["joined_submission_elapsed"])
            results[device][key].append((full_file_size / 1_048_576) / joined_runtime_seconds)

            if key not in reconfiguration_results[device]:
                reconfiguration_results[device][key] = []

            reconfiguration_runtime_seconds = float(data["overall_submission_elapsed"]) + float(data["ctx_stop_elapsed"])
            reconfiguration_runtime_seconds = max(joined_runtime_seconds, reconfiguration_runtime_seconds)
            reconfiguration_results[device][key].append((full_file_size / 1_048_576) / reconfiguration_runtime_seconds)

    averaged_results[device] = {key: {"static": 0, "reconfiguration": 0} for key in results[device].keys()}
    for key in results[device].keys():
        key_avg = sum(results[device][key]) / len(results[device][key])
        averaged_results[device][key]["static"] = key_avg
        reconfiguration_key_avg = sum(reconfiguration_results[device][key]) / len(reconfiguration_results[device][key])
        averaged_results[device][key]["reconfiguration"] = reconfiguration_key_avg

display(averaged_results["bf3"])

# Write to CSV
with open(f"../../tex/figures/za-evaluation/coprocessing-decompress-lz4-r.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write header
    writer.writerow(["sharing_percentage", "static", "reconfiguration"])

    # Write rows
    sorted_keys = sorted(averaged_results["bf3"].keys())
    for key in sorted_keys: # only bf3 has this item
        writer.writerow([key, averaged_results["bf3"][key]["static"], averaged_results["bf3"][key]["reconfiguration"]])


{40: {'static': 9680.359366689136, 'reconfiguration': 164.50570772535045},
 30: {'static': 8526.014723419525, 'reconfiguration': 164.52023381052172},
 10: {'static': 7349.998844811561, 'reconfiguration': 164.02470324891502},
 20: {'static': 8055.9290149362, 'reconfiguration': 164.53369447794847},
 50: {'static': 11173.193967907226, 'reconfiguration': 163.09291234801472},
 90: {'static': 26920.13681904299, 'reconfiguration': 162.05004642094357},
 80: {'static': 20043.318426500908, 'reconfiguration': 162.87624556857142},
 70: {'static': 15111.081790101607, 'reconfiguration': 163.8558019412535},
 60: {'static': 12909.527052781736, 'reconfiguration': 163.5658564003557},
 100: {'static': 26565.192144620378, 'reconfiguration': 163.57776057104434}}